# ДЗ 4

1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention
2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.
3. сделать разбиение набора данных не тренировочную и тестовую выборки
4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)
5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями
6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей
7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево
8. (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion
9. (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево
Ссылки
https://towardsdatascience.com/a-quick-uplift-modeling-introduction-6e14de32bfe0
https://habr.com/ru/company/ru_mts/blog/485980/#reference1
https://en.wikipedia.org/wiki/Uplift_modelling
https://www.youtube.com/watch?v=yFQAIJBYXI0
https://www.youtube.com/watch?v=jCUcYiBK03I
https://www.uplift-modeling.com/en/latest/
https://arxiv.org/pdf/1809.04559.pdf
https://catboost.ai/docs/concepts/about.html

Библиотеки и пакеты
causalml
sklift
catboost

In [80]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import itertools

from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split

In [81]:
df = pd.read_csv('HW_data.csv', ',')
df

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0
...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web,Discount,0
63996,5,38.91,0,1,Urban,1,Phone,Discount,0
63997,6,29.99,1,0,Urban,1,Phone,Discount,0
63998,1,552.94,1,0,Surburban,1,Multichannel,Buy One Get One,0


In [82]:
df.rename(columns={'offer': 'treatment', 'conversion' : 'target'}, inplace=True)
df

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0
...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web,Discount,0
63996,5,38.91,0,1,Urban,1,Phone,Discount,0
63997,6,29.99,1,0,Urban,1,Phone,Discount,0
63998,1,552.94,1,0,Surburban,1,Multichannel,Buy One Get One,0


In [83]:
df['treatment'].value_counts()

Buy One Get One    21387
Discount           21307
No Offer           21306
Name: treatment, dtype: int64

In [84]:
to_binary = {'No Offer': 0, 'Buy One Get One': 1, 'Discount' : 1}
df['treatment'] = df['treatment'].replace(to_binary)
df

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
3,9,675.83,1,0,Rural,1,Web,1,0
4,2,45.34,1,0,Urban,0,Web,1,0
...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web,1,0
63996,5,38.91,0,1,Urban,1,Phone,1,0
63997,6,29.99,1,0,Urban,1,Phone,1,0
63998,1,552.94,1,0,Surburban,1,Multichannel,1,0


In [85]:
#indices_learn, indices_valid = train_test_split(df.index, test_size=0.3, random_state=2)
# X_train, X_test, y_train, y_test = train_test_split(df.drop('target', 1), 
#                                                     df['target'], random_state=2)
# indexes_train = df_train.index
# indexes_test = df_test.index
# indexes_learn, indices_valid = train_test_split(df.index, test_size=0.3, random_state=123)

In [86]:
# X_train = df_features.loc[indices_learn, :] # 
# y_train = df.loc[indices_learn, 'target']
# treat_train = df_train.loc[indices_learn, 'treatment_flg']

In [87]:
X_train, X_test, y_train, y_test, treat_train, treat_test = train_test_split(df.drop(columns=['target']), 
                                                            df['target'],
                                                            df['treatment'], test_size=0.3,
                                                            random_state=2)

In [88]:
X_train.shape

(44800, 8)

## 4.

recency - months since last purchase
history - $value of the historical purchases
used_discount - indicates if the customer used a discount before
used_bogo - indicates if the customer used a buy one get one before
zip_code - class of the zip code as Suburban/Urban/Rural
is_referral - indicates if the customer was acquired from referral channel
channel - channels that the customer using, Phone/Web/Multichannel

In [89]:
# Если последняя покупка была совершена недавно - ставим 1, если более 1 месяца назад - ставим 0
def long_ago (df):
    df['long_ago'] = 1 # По дефолту ставим в новом столбце 1 всем
    df.loc[(df['recency'] > 1), 'long_ago'] = 0 # Всем, у кого покупка была более 1 мес назад ставим 0 
    return df

In [90]:
X_train = long_ago(X_train)
X_test = long_ago(X_test)

In [91]:
X_test

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,long_ago
44835,1,328.09,1,1,Rural,0,Web,0,1
37973,7,76.44,0,1,Surburban,0,Phone,1,0
16137,3,140.68,0,1,Surburban,0,Phone,0,0
18901,9,29.99,1,0,Rural,1,Phone,1,0
26736,1,564.19,0,1,Surburban,1,Multichannel,1,1
...,...,...,...,...,...,...,...,...,...
9840,5,613.55,0,1,Urban,1,Multichannel,1,0
22492,9,214.56,0,1,Rural,0,Phone,1,0
3113,10,108.15,0,1,Rural,0,Phone,1,0
29452,7,164.97,0,1,Urban,1,Phone,1,0


## 5.

In [92]:
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel

# sklift поддерживает любые модели, 
# которые удовлетворяют соглашениями scikit-learn
# Для примера воспользуемся catboost
from catboost import CatBoostClassifier

In [93]:
cat_features = ['zip_code', 'channel']

In [94]:
sm = SoloModel(CatBoostClassifier(iterations=20, thread_count=2, random_state=2, silent=True))
sm = sm.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_sm = sm.predict(X_test)

In [95]:
models_results = {
    'approach': [],
    'uplift@30%': []
}

In [96]:
sm_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.3)
print(f'uplift@30%: {sm_score:.4f}')

models_results['approach'].append('SoloModel')
models_results['uplift@30%'].append(sm_score)

uplift@30%: 0.0768


In [97]:
models_results

{'approach': ['SoloModel'], 'uplift@30%': [0.07681899463971034]}

In [98]:
total_table = pd.DataFrame(columns=['approach','uplift@10%', 'uplift@20%'])

In [99]:
sm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.1)

In [100]:
sm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.2)

In [101]:
total_table = total_table.append(
    { 'approach' : 'S learner', 'uplift@10%': sm_score_10, 'uplift@20%': sm_score_20}, 
    ignore_index=True)

In [102]:
from sklift.models import ClassTransformation

In [103]:
ct = ClassTransformation(CatBoostClassifier(iterations=20, thread_count=2, random_state=2, silent=True))
ct = ct.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_ct = ct.predict(X_test)

ct_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.1)
ct_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.2)
total_table = total_table.append(
    { 'approach' : 'Transformation', 'uplift@10%': ct_score_10, 'uplift@20%': ct_score_20},
    ignore_index=True)

<ipython-input-103-2ceadf8c57ca>:2: UserWarning: It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.
  ct = ct.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})


In [104]:
total_table

,approach,uplift@10%,uplift@20%
0,S learner,0.102681,0.088192
1,Transformation,0.253916,0.227348


In [105]:
from sklift.models import TwoModels

tm = TwoModels(
    estimator_trmnt=CatBoostClassifier(iterations=20, thread_count=2, random_state=2, silent=True), 
    estimator_ctrl=CatBoostClassifier(iterations=20, thread_count=2, random_state=2, silent=True), 
    method='vanilla'
)
tm = tm.fit(
    X_train, y_train, treat_train,
    estimator_trmnt_fit_params={'cat_features': cat_features}, 
    estimator_ctrl_fit_params={'cat_features': cat_features}
)

uplift_tm = tm.predict(X_test)

tm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.1)
tm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.2)

total_table = total_table.append(
    { 'approach' : 'Two models', 'uplift@10%': tm_score_10, 'uplift@20%': tm_score_20},
    ignore_index=True)

In [106]:
total_table

,approach,uplift@10%,uplift@20%
0,S learner,0.102681,0.088192
1,Transformation,0.253916,0.227348
2,Two models,0.108094,0.089655


In [109]:
%%time
from IPython.display import Image
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

uplift_model = UpliftTreeClassifier(max_depth=8, min_samples_leaf=200, min_samples_treatment=50,
                                    n_reg=100, evaluationFunction='KL', control_name='control')

uplift_model.fit(X_train_tree.values,
                 treatment=treat_train.map({1: 'treatment1', 0: 'control'}).values,
                 y=y_train)

graph = uplift_tree_plot(uplift_model.fitted_uplift_tree, features)
Image(graph.create_png())

ModuleNotFoundError: No module named 'causalml.inference'

Не ставиться causalml без ошибок...
linux Mint, python 3.8